<a href="https://colab.research.google.com/github/Tcrista/SVHN/blob/master/svhn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import scipy.io
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers.normalization import BatchNormalization
from keras import backend as K

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
batch_size = 1000
num_classes = 10
epochs = 25

In [0]:
train = scipy.io.loadmat('/content/drive/My Drive/Colab Notebooks/train_32x32')
X_train = train['X']
y_train = train['y']

test = scipy.io.loadmat('/content/drive/My Drive/Colab Notebooks/test_32x32')
X_test = test['X']
y_test = test['y']

img_rows, img_cols = 32, 32

In [0]:
X_train_new = np.empty([img_rows, img_cols, 1, X_train.shape[-1]])
for i in range(X_train.shape[-1]):
    X_train_new[:,:,:,i] = cv2.cvtColor(X_train[:,:,:,i], cv2.COLOR_BGR2GRAY).reshape(img_rows, img_cols, 1)
X_train = X_train_new
del X_train_new

X_test_new = np.empty([img_rows, img_cols, 1, X_test.shape[-1]])
for i in range(X_test.shape[-1]):
    X_test_new[:,:,:,i] = cv2.cvtColor(X_test[:,:,:,i], cv2.COLOR_BGR2GRAY).reshape(img_rows, img_cols, 1)
X_test = X_test_new
del X_test_new

In [0]:
if K.image_data_format() == 'channels_first':
    X_train = np.moveaxis(X_train, -1, 0)
    X_train = np.moveaxis(X_train, -1, 1)
    X_test = np.moveaxis(X_test, -1, 0)
    X_test = np.moveaxis(X_test, -1, 1)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = np.moveaxis(X_train, -1, 0)
    X_test = np.moveaxis(X_test, -1, 0)
    input_shape = (img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [0]:
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
model = Sequential()
model.add(Conv2D(16, kernel_size=5, input_shape=input_shape, padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(32, 5, padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(64, 5, padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

lr = 0.001
optimizers = {"SGD": keras.optimizers.SGD(lr=lr), "RMSprop": keras.optimizers.RMSprop(lr=lr),
              "Adadelta": keras.optimizers.Adadelta(lr=lr), "Adam": keras.optimizers.Adam(lr=lr)}

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizers["Adam"],
              metrics=['accuracy'])













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 73257 samples, validate on 26032 samples
Epoch 1/25
73257/73257 [==============================] - 8s 108us/step - loss: 2.5964 - acc: 0.1689 - val_loss: 2.2161 - val_acc: 0.1942
Epoch 2/25
73257/73257 [==============================] - 6s 76us/step - loss: 2.1624 - acc: 0.2198 - val_loss: 2.2781 - val_acc: 0.1716
Epoch 3/25
73257/73257 [==============================] - 6s 76us/step - loss: 1.7055 - acc: 0.4048 - val_loss: 1.5622 - val_acc: 0.4869
Epoch 4/25
73257/73257 [==============================] - 6s 76us/step - loss: 1.2276 - acc: 0.5912 - val_loss: 1.1600 - val_acc: 0.6091
Epoch 5/25
73257/73257 [==============================] - 6s 75us/step - loss: 0.8824 - acc: 0.7163 - val_loss: 0.9597 - val_acc: 0.6912
Epoch 6/25
73257/73257 [==============================] - 6s 75us/step - loss: 0.7253 - acc: 0.7715 - val_loss: 0.7251 - val_acc: 0.7672
Epoch 7/25
73257/73257 [=======

In [0]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.2978749904912091
Test accuracy: 0.9137215734480639
